In [2]:
import pandas as pd

parsed_df = pd.read_excel('parsed.xlsx')
parsed_df

,drone_model,dataset,controller,source_file,date,time,message
0,DJI_Inspire_1,df010,mobile_iOS,336-80799827e1156d2c54438d9e04b6100afbaad8e3,2017-06-29,15:24:31,Taking Off
1,DJI_Inspire_1,df010,mobile_iOS,336-80799827e1156d2c54438d9e04b6100afbaad8e3,2017-06-29,15:24:31,Taking Off
2,DJI_Inspire_1,df010,mobile_iOS,336-80799827e1156d2c54438d9e04b6100afbaad8e3,2017-06-29,15:24:32,Taking Off
3,DJI_Inspire_1,df010,mobile_iOS,336-80799827e1156d2c54438d9e04b6100afbaad8e3,2017-06-29,15:24:32,Taking Off
4,DJI_Inspire_1,df010,mobile_iOS,336-80799827e1156d2c54438d9e04b6100afbaad8e3,2017-06-29,15:24:32,Home Point Recorded. RTH Altitude: 30m.
...,...,...,...,...,...,...,...
1841,DJI_Spark,df008,mobile_iOS_backup,433-9e6d8a9d4477a3e21f27bcc82e8ac2de739e706d,2017-06-29,13:31:08,Obstacle Avoidance will be disabled in landing.
1842,DJI_Spark,df008,mobile_iOS_backup,433-9e6d8a9d4477a3e21f27bcc82e8ac2de739e706d,2017-06-29,13:31:34,The remaining battery is only enough for RTH. ...
1843,DJI_Spark,df008,mobile_iOS,360-8d693c3df81df0a633759d207b11bb4a711432fb,2018-06-19,12:07:40,Home Point Recorded. RTH Altitude: 30m.
1844,DJI_Spark,df008,mobile_iOS,360-8d693c3df81df0a633759d207b11bb4a711432fb,2018-06-19,12:09:10,Fly with caution and ensure the aircraft remai...


In [6]:
import pandas as pd
from pathlib import Path

def analyze_prior_work(df, output_dir="reports"):
    """
    Analyzes prior work CSV and generates:
    1. Message counts per source file
    2. Comparison with current VTO inventory
    """
    # Load and clean data
    # df = pd.read_excel(prior_path)
    
    # Basic cleaning
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['message'])  # Remove empty messages
    
    # Generate report 1: Messages per source file
    file_report = df.groupby(['drone_model', 'dataset', 'controller', 'source_file']).agg(
        message_count=('message', 'size'),
        date_range=('date', lambda x: f"{x.min().date()} to {x.max().date()}"),
        time_range=('time', lambda x: f"{x.iloc[0]} - {x.iloc[-1]}")
    ).reset_index()
    
    # Generate report 2: Summary by drone model and controller
    summary_report = df.groupby(['drone_model', 'controller']).agg(
        unique_files=('source_file', 'nunique'),
        total_messages=('message', 'size'),
        date_range=('date', lambda x: f"{x.min().date()} to {x.max().date()}")
    ).reset_index()
    
    # Save reports
    Path(output_dir).mkdir(exist_ok=True)
    with pd.ExcelWriter(f"{output_dir}/prior_work_analysis.xlsx") as writer:
        file_report.to_excel(writer, sheet_name="Per File Counts", index=False)
        summary_report.to_excel(writer, sheet_name="Model Summary", index=False)
        df.to_excel(writer, sheet_name="Raw Data Sample", index=False)
    
    print(f"Analysis complete. Reports saved to {output_dir}/prior_work_analysis.xlsx")
    return file_report, summary_report

def compare_with_current(prior_counts, current_inventory_path):
    """
    Compares prior work counts with current VTO inventory
    """
    current = pd.read_excel(current_inventory_path, sheet_name="Detailed List")
    
    comparison = pd.merge(
        prior_counts.rename(columns={'source_file': 'FileName'}),
        current[['FileName', 'Path', 'DeviceType']],
        on='FileName',
        how='outer',
        indicator=True
    )
    
    # Categorize results
    comparison['status'] = comparison['_merge'].map({
        'left_only': "Only in prior work",
        'right_only': "Only in current",
        'both': "In both"
    })
    
    return comparison.drop(columns=['_merge'])

if __name__ == "__main__":
    # Example usage
    priod_df = pd.read_csv('parsed-cleansed.csv')
    current_inventory = "vto_flightlog_inventory.xlsx"
    
    # Step 1: Analyze prior work
    file_counts, model_summary = analyze_prior_work(priod_df)
    
    # Step 2: Compare with current (if available)
    # if Path(current_inventory).exists():
    #     comparison = compare_with_current(file_counts, current_inventory)
    #     comparison.to_excel("reports/version_comparison.xlsx", index=False)
    #     print("\nComparison with current inventory:")
    #     print(comparison['status'].value_counts())

Analysis complete. Reports saved to reports/prior_work_analysis.xlsx


In [ ]:
file_counts

1853

In [8]:
model_summary

,drone_model,controller,unique_files,total_messages,date_range
0,DJI_Inspire_1,mobile_android,4,52,2017-03-08 to 2018-06-20
1,DJI_Inspire_1,mobile_iOS,4,54,2017-06-29 to 2017-08-03
2,DJI_Inspire_2,mobile_android_logical,2,214,2018-06-20 to 2018-06-20
3,DJI_Inspire_2,mobile_android_physical,6,159,2017-08-29 to 2017-08-29
4,DJI_Inspire_2,mobile_iOS_backup,8,68,2017-08-29 to 2018-06-20
5,DJI_Matrice_210,controller_tablet_intact,10,34,2018-06-14 to 2018-06-21
6,DJI_Matrice_210,table_physical_intact,5,49,2018-06-14 to 2018-06-20
7,DJI_Matrice_600,mobile_android_logical,2,90,2018-06-20 to 2018-06-20
8,DJI_Matrice_600,mobile_iOS_backup,1,6,2018-06-20 to 2018-06-20
9,DJI_Mavic_2,mobile_android,10,136,2017-10-10 to 2018-09-26


In [9]:
model_summary['total_messages'].sum()

1853